In [ ]:
import SimpleITK as sitk
import numpy as np
import tensorflow as tf
import math
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
def plot_histogram(data_col, title, xlabel, ylabel, x_range=None):
    """
    Plot histogram showing the distribution of patient ages.

    Parameters:
    data (Series): Pandas Series containing patient ages.
    """
    plt.figure(figsize=(10, 6))
    sns.histplot(data_col, bins=20, kde=True, color='skyblue')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if x_range:
        plt.xlim(x_range)
    plt.show();

# Image Data Preprocessing

In [ ]:
def forresnet(image_sample):
    """
    Preprocess the image for ResNet-50 model

    """
    # Assuming images[0] is a SimpleITK image
    image_sitk = image_sample


    # # Selecting a single slice (e.g., the first slice)
    # single_slice = image_sitk[0, :, :]

    # Convert SimpleITK image to numpy array
    image_array = sitk.GetArrayFromImage(image_sitk)  #single_slice
    slice_index = 0
    image_array = image_array[slice_index, :, :]
    # print("Original shape:", image_array.shape)  # (23, 384)

    # Reshape to add an extra dimension
    image_array = np.expand_dims(image_array, axis=-1)
    # print("Shape after adding dimension:", image_array.shape)  # (23, 384, 1)

    # Resize the image to (224, 224)
    image_array = tf.image.resize(image_array, (224, 224))
    # print("Resized shape:", image_array.shape)  # (224, 224, 1)

    # Remove the extra dimension
    image_array = tf.squeeze(image_array, axis=-1)
    # print("Shape after squeezing:", image_array.shape)  # (224, 224)

    # Convert to grayscale NOT NEEDED BECAUSE IT IS ALREADY IN GRAYSCALE
    # image_array = tf.image.rgb_to_grayscale(image_array)
    # print("Shape after converting to grayscale:", image_array.shape)  # (224, 224, 1)

    # Stack to create a 3-channel image
    image_array = tf.stack([image_array] * 3, axis=-1)
    # print("Shape after stacking:", image_array.shape)  # (224, 224, 3)

    # Normalize the image
    # max_value = tf.reduce_max(image_array)
    # image_array = image_array / max_value
    image_array = image_array / 255

    # print("Shape after normalization:", image_array.shape)

    # # Add batch dimension
    # image_array = tf.expand_dims(image_array, axis=0)
    # print("Final shape with batch dimension:", image_array.shape)
    
    # print("Minimum value:", tf.reduce_min(image_array))
    # print("Maximunm value:", tf.reduce_max(image_array))
    # print('-------------------------')
    

    return image_array

# Model Evalution

In [ ]:
def evaluate_model(model, threshold, X_train, y_train, X_val, y_val, X_test=None, y_test=None, include_test=False):
    def calculate_metrics(X, y):
        # Predict
        y_pred = model.predict(X)
        y_pred_binary = (y_pred > threshold).astype('int32')  # Convert probabilities to binary predictions

        # Calculate metrics
        accuracy = accuracy_score(y, y_pred_binary)
        precision = precision_score(y, y_pred_binary, zero_division=0)
        recall = recall_score(y, y_pred_binary)
        f1 = f1_score(y, y_pred_binary)
        conf_matrix = confusion_matrix(y, y_pred_binary)

        # Find correctly and wrongly predicted indices
        correct_predictions = [i for i in range(len(y)) if y_pred_binary[i] == y[i]]
        wrong_predictions = [i for i in range(len(y)) if y_pred_binary[i] != y[i]]

        return accuracy, precision, recall, f1, conf_matrix, correct_predictions, wrong_predictions

    # Calculate metrics for train and validation sets
    train_metrics = calculate_metrics(X_train, y_train)
    val_metrics = calculate_metrics(X_val, y_val)

    # If include_test is True, calculate metrics for the test set
    test_metrics = None
    if include_test:
        test_metrics = calculate_metrics(X_test, y_test)

    # Create lists for correct and wrong predictions
    correct_predictions = []
    wrong_predictions = []

    # Add train, validation, and test predictions to the lists
    if train_metrics[-2] is not None:
        correct_predictions.append(train_metrics[-2])
        wrong_predictions.append(train_metrics[-1])

    if val_metrics[-2] is not None:
        correct_predictions.append(val_metrics[-2])
        wrong_predictions.append(val_metrics[-1])

    if test_metrics[-2] is not None:
        correct_predictions.append(test_metrics[-2])
        wrong_predictions.append(test_metrics[-1])

    # Create a DataFrame for metrics
    metrics_df = pd.DataFrame([train_metrics, val_metrics, test_metrics],
                              columns=['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Confusion Matrix',
                                       'Correct Predictions', 'Wrong Predictions'])

    # Add a column for set type
    metrics_df['Set Type'] = ['Train', 'Validation', 'Test']
    metrics_df = metrics_df.set_index('Set Type')

    # Remove the confusion matrix, correct predictions, and wrong predictions columns from the DataFrame
    metrics_df = metrics_df.drop(['Confusion Matrix', 'Correct Predictions', 'Wrong Predictions'], axis=1)

    # Filter out the test row if include_test is False
    if not include_test:
        metrics_df = metrics_df.drop('Test', axis=0)

    display(metrics_df)

    ### Plot confusion matrices
    if include_test == False:
        conf_matrices = [train_metrics[4], val_metrics[4]]
        titles = ['Train', 'Validation']
        num_plots = 2
    else:
        conf_matrices = [train_metrics[4], val_metrics[4], test_metrics[4]]
        titles = ['Train', 'Validation', 'Test']
        num_plots = 3

    fig, axes = plt.subplots(1, num_plots, figsize=(5 * num_plots, 5))

    for i, (conf_matrix, title) in enumerate(zip(conf_matrices, titles)):
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[i])
        axes[i].set_title(f'Confusion Matrix - {title}')

    plt.tight_layout()
    plt.show()

    return metrics_df, correct_predictions, wrong_predictions


In [ ]:
def grad_cam(model, img_arrays, layer_name):

    #gradient tape context to record the forward pass
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])

    #initialize a list to store CAMs for each image
    cams = []

    for img_array in img_arrays:
        #get the last convolutional layer and the output layer of the model
        with tf.GradientTape() as tape:
            conv_output, predictions = grad_model(img_array)
            # Get the index of the top prediction
            class_index = np.argmax(predictions[0])
            # Fetch the gradient of the top predicted class with respect to the output feature map of the last convolutional layer
            gradient = tape.gradient(predictions[:, class_index], conv_output)

        #global average pooling to get the weights of each feature map
        weights = tf.reduce_mean(gradient, axis=(1, 2))

        #get the output of the last convolutional layer
        conv_output = conv_output[0]

        #generate the class activation map
        cam = np.ones(conv_output.shape[:2], dtype=np.float32)  # Ensure cam has only 2 dimensions

        #reshape the weights to match the spatial dimensions of the convolutional layer output
        weights = tf.reshape(weights, (-1, 1, 1))

        #multiply each feature map by its corresponding weight and sum them up
        for i, w in enumerate(weights):
            cam += w * conv_output[:, :, i]

        #normalize the CAM
        cam = cv2.resize(cam.numpy(), (224, 224))
        cam = np.maximum(cam, 0)
        cam_max = cam.max()
        if cam_max != 0:
            cam = cam / cam_max
            
        cams.append(cam)

    return cams


In [ ]:
def visualize_gradcam(model, layer_name, X_images, indices, which_set, y_train, y_val, y_test, num_images=5):
    
    correct_imgs = [X_images[i] for i in indices]  # Load images

    if which_set == 'train':
        correct_labels = ['cancer' if y_train[i] == 1 else 'no cancer' for i in indices]
    elif which_set == 'val':
        correct_labels = ['cancer' if y_val[i] == 1 else 'no cancer' for i in indices]
    elif which_set == 'test':
        correct_labels = ['cancer' if y_test[i] == 1 else 'no cancer' for i in indices]


    # Expand dimensions for each image
    img_arrays = [np.expand_dims(img, axis=0) for img in correct_imgs]

    # Generate GradCAMs
    cams = grad_cam(model, img_arrays, layer_name)

    # Display the original images and their GradCAM overlays
    fig, axes = plt.subplots(2, num_images, figsize=(20, 8))

    for i, (img, cam, correct_label) in enumerate(zip(correct_imgs, cams, correct_labels)):
        axes[0, i].imshow(img)
        axes[0, i].axis('off')
        axes[0, i].set_title('Original')

        axes[1, i].imshow(img)
        axes[1, i].imshow(cam, cmap='jet', alpha=0.5)
        axes[1, i].axis('off')
        axes[1, i].set_title(f'GradCAM Overlay (True: {correct_label})')

    plt.tight_layout()
    plt.show()
